# Тема “ Построение надежных схем валидации решения, оптимизация целевых метрик.”


<b>Основное задание:
<ul>Даны выборки для обучения и для тестирования
<li>Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче.
<li>Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud.
<li>Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые.   
<li>Внимание! выборка assignment_2_test.csv - наш аналог лидерборда.   
<li>Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. 
</ul>

***Для других целей использовать выборку запрещено!.***
<hr>
<ul>
Терминалогия, используемая в задании:
<li>обучающая выборка - выборка, которая передается в метод fit / train;
<li>валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
<li>тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
<li>ЛБ - лидерборд, выборка assignment_2_test.csv.
</ul>
    <hr>
Задания:
<ol>
<li><a href="#task_1">Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.
</a>
<li><a href = "#task_2">Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.
</a>
<li><a href = "#task_3">Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.
</a>
<li><a href = "#task_4">Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.
</a>
<li><a href = "#task_5">Задание 5: сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.
</a>
<li><a href = "#task_6">Задание 6 (опциональное): сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.
</a>
<li><a href = "#task_7">Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).
</a>
</ol>

## Импорт библиотек

In [1]:
import warnings
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score

from src_utils import optimization_memory_usage, base_describe

warnings.simplefilter("ignore")
%matplotlib inline

## Загрузка данных

In [2]:
train_df = pd.read_csv('../data/assignment_2_train.csv')
test_df = pd.read_csv('../data/assignment_2_test.csv')

In [3]:
statistic_train = base_describe(data=train_df, data_name='train')
statistic_test= base_describe(data=test_df, data_name='test')

statistic_df = pd.concat([statistic_train, statistic_test], axis=1)
statistic_df

,train,test
Количество наблюдений,180000,100001
Количество признаков,394,394
Признаков float64,376,376
Признаков object,14,14
Признаков int64,4,4
Признаков с NaN,319,299
Дискретных признаков,173,208
Непрерывных признаков,207,172


In [4]:
train_df = optimization_memory_usage(train_df)

Memory usage of dataframe is 541.08 MB
Memory usage after optimization is: 158.10 MB
Decreased by 70.8%


In [5]:
test_df = optimization_memory_usage(test_df)

Memory usage of dataframe is 300.60 MB
Memory usage after optimization is: 83.45 MB
Decreased by 72.2%


In [6]:
TARGET_NAME = 'isFraud'
DROP_NAMES = ['TransactionID']

## Выполнение заданий

<p><a name="task_1"></a></p>

### Задание 1: сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.


<p><a name="task_2"></a></p>

### Задание 2: сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер 

<p><a name="task_3"></a></p>

### Задание 3: построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.


<p><a name="task_4"></a></p>

### Задание 4: выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.


<p><a name="task_5"></a></p>

### Задание 5: сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

<p><a name="task_6"></a></p>

### Задание 6 (опциональное): сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.


<p><a name="task_7"></a></p>

### Задание 7 (совсем опциональное): в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).
